In [65]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Thread
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse

In [66]:
# Load the Cornell Box
Thread.thread().file_resolver().append('cbox')
scene = load_file('cbox/cbox.xml')

# Find differentiable scene parameters
params = traverse(scene)
print(params)

2021-02-23 14:09:46 INFO main [xml.cpp:1221] Loading XML file "cbox/cbox.xml" ..

2021-02-23 14:09:46 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-02-23 14:09:46 INFO main [xml.cpp:354] "cbox/cbox.xml": in-memory version upgrade (v0.4.0 -> v2.2.1) ..

2021-02-23 14:09:46 DEBUG main [OBJMesh] Loading mesh from "cbox_luminaire.obj" ..

2021-02-23 14:09:46 DEBUG main [OBJMesh] "cbox_luminaire.obj": read 2 faces, 4 vertices (120 B in 0ms)

...

2021-02-23 14:09:46 DEBUG main [OBJMesh] "cbox_smallbox.obj": read 12 faces, 24 vertices (720 B in 0ms)

2021-02-23 14:09:46 DEBUG main [OBJMesh] Loading mesh from "cbox_largebox.obj" ..

2021-02-23 14:09:46 DEBUG main [OBJMesh] "cbox_largebox.obj": read 12 faces, 24 vertices (720 B in 0ms)

2021-02-23 14:09:46 INFO main [Scene] Building scene in OptiX ..

ParameterMap[
    PerspectiveCamera.near_clip,
    PerspectiveCamera.far_clip,
    PerspectiveCamera.focus_distance,
    PerspectiveCamera.shutter_open,
    PerspectiveCamera.shutter_open_time,
  * box.reflectance.value,
  * white.reflectance.value,
  * red.reflectance.value,
  * green.reflectance.value,
  * light.reflectance.value,
    OBJMesh.to_world,
  * OBJMesh.emitter.radiance.value,
    OBJMesh.vertex_count,
    OBJMesh.face_count,
  * OBJMesh.faces_buf,
  * OBJMesh.vertex_positions_buf,
  * OBJMesh.vertex_normals_buf,
  * OBJMesh.vertex_texcoords_buf,
    OBJMesh_1.to_world,
    OBJMesh_1.vertex_count,
    OBJMesh_1.face_count,
  * OBJMesh_1.faces_buf,
  * OBJMesh_1.vertex_positions_buf,
  * OBJMesh_1.vertex_normals_buf,
  * OBJMesh_1.vertex_texcoords_buf,
    OBJMesh_2.to_world,
    OBJMesh_2.vertex_count,
    OBJMesh_2.face_count,
  * OBJMesh_2.faces_buf,
  * OBJMesh_2.vertex_positions_buf,
  * OBJMesh_2.vertex_normals_buf,
  * OBJMesh_2.vertex_texcoords_buf,
    OBJMesh_3.to

In [89]:
params.keep(['red.reflectance.value', 'green.reflectance.value'])

In [90]:
from mitsuba.core import Color3f
param_ref = Color3f(params['red.reflectance.value'])
param_ref_2 = Color3f(params['green.reflectance.value'])

In [91]:
print(param_ref, param_ref_2)

[[0.552138, 0.0352953, 0.013765]] [[0.144489, 0.265683, 0.0950349]]


In [70]:
# Render a reference image (no derivatives used yet)
from mitsuba.python.autodiff import render, write_bitmap
image_ref = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('imgs/out_ref.png', image_ref, crop_size)

In [96]:
# Change the left wall into a bright white surface
params['red.reflectance.value'] = [.9, .9, .9]
params['green.reflectance.value'] = [.0, .0, .0]
#params['light.reflectance.value'] = [-1., -1., -1.]
params.update()

In [97]:
# Construct an Adam optimizer that will adjust the parameters 'params'
from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=.2)

In [98]:
for it in range(100):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    write_bitmap('imgs/out_%03i.png' % it, image, crop_size)
    
    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()
    
    err_ref = ek.hsum(ek.sqr(param_ref - params['red.reflectance.value']) + ek.sqr(param_ref_2 - params['green.reflectance.value']))
    print('Iteration %03i: error=%g' % (it, err_ref[0]))

Iteration 000: error=0.952258
Iteration 001: error=0.478305
Iteration 002: error=0.256477
Iteration 003: error=0.139167
Iteration 004: error=0.0854785
Iteration 005: error=0.0414775
Iteration 006: error=0.0132967
Iteration 007: error=0.00653025
Iteration 008: error=0.0160117
Iteration 009: error=0.0288042
Iteration 010: error=0.0357192
Iteration 011: error=0.0342297
Iteration 012: error=0.0309044
Iteration 013: error=0.0299802
Iteration 014: error=0.0334587
Iteration 015: error=0.0368352
Iteration 016: error=0.0369216
Iteration 017: error=0.0329182
Iteration 018: error=0.0255969
Iteration 019: error=0.0168737
Iteration 020: error=0.00941885
Iteration 021: error=0.00601282
Iteration 022: error=0.00795854
Iteration 023: error=0.0129977
Iteration 024: error=0.0201676
Iteration 025: error=0.0277682
Iteration 026: error=0.0327679
Iteration 027: error=0.0327826
Iteration 028: error=0.0282574
Iteration 029: error=0.0230143
Iteration 030: error=0.0188958
Iteration 031: error=0.0158603
Iteratio